In [1]:
%load_ext autoreload
%autoreload 2

import os

import mlflow
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain_anthropic import ChatAnthropic
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

from killtherobot_dev.prompts import prompts

_ = load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

mlflow.set_experiment("LangChain Tracing")
client = mlflow.tracking.MlflowClient()
mlflow.langchain.autolog(
    log_models=True,
    log_input_examples=True,
    silent=True,
)


def get_llm(model_name):
    match model_name:
        case "gpt-4o":
            return ChatOpenAI(
                model="gpt-4o",
                api_key=OPENAI_API_KEY,
                temperature=0.9,
            )
        case "claude-3-5-sonnet":
            return ChatAnthropic(
                model="claude-3-5-sonnet-20240620",
                api_key=ANTHROPIC_API_KEY,
                temperature=0.9,
            )
        case _:
            raise ValueError(f"Model {model_name} not found")


good_qa_pairs = ""


def get_answer(bot, question, good_qa_pairs, llm_name):
    llm = get_llm(llm_name)
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", prompts[bot]["system"]),
            ("human", prompts[bot]["prompt"]),
        ]
    )
    answer_chain = prompt | llm | StrOutputParser()
    answer = answer_chain.invoke({"question": question, "good_qa_pairs": good_qa_pairs})

    trace = mlflow.get_last_active_trace()
    client.set_trace_tag(trace.info.request_id, "task", "answer_prompt")
    client.set_trace_tag(trace.info.request_id, "bot", bot)
    client.set_trace_tag(trace.info.request_id, "llm", llm.model_name)
    return answer

In [2]:
questions = [
    "Before becoming a superhero, Tony Stark spent his days __.",
    "Nick Fury interrupted __ to recruit the newest Avenger.",
    "Peter Parker initially refused to join the Avengers because __.",
    "Doctor Strange learned the mystic arts from __ in Kamar-Taj.",
    "To prove their worth, the new X-Men recruit had to __.",
    "During their first mission, the Guardians of the Galaxy accidentally __.",
    "Before confronting Thanos, the Avengers prepared by __.",
    "In their darkest hour, the X-Men were forced to __.",
    "After defeating Ultron, Tony Stark celebrated by __.",
    "While returning to Earth, Captain Marvel encountered __.",
    "To save the universe, Doctor Strange had to sacrifice __.",
    "After saving the multiverse, Spider-Man's greatest reward was __.",
]

In [4]:
for i, q in enumerate(questions):
    answer = get_answer("sarcastic", q, good_qa_pairs, "gpt-4o")
    print(f"{i+1}. {q} \n\n {answer}\n\n")

1. Before becoming a superhero, Tony Stark spent his days __. 

 betting on himself


2. Nick Fury interrupted __ to recruit the newest Avenger. 

 a TED Talk marathon


3. Peter Parker initially refused to join the Avengers because __. 

 "high school algebra."


4. Doctor Strange learned the mystic arts from __ in Kamar-Taj. 

 YouTube meditation channel


5. To prove their worth, the new X-Men recruit had to __. 

 out-sarcasm Wolverine


6. During their first mission, the Guardians of the Galaxy accidentally __. 

 misplaced the universe


7. Before confronting Thanos, the Avengers prepared by __. 

 Googling "retirement plans."


8. In their darkest hour, the X-Men were forced to __. 

 watch cat videos


9. After defeating Ultron, Tony Stark celebrated by __. 

 upgrading his sarcasm


10. While returning to Earth, Captain Marvel encountered __. 

 existential dread therapy


11. To save the universe, Doctor Strange had to sacrifice __. 

 Marvel's last storyline


12. After savi